In [2]:
import pandas as pd
import numpy as np
import os

In [4]:
os.chdir(r'/Users/hongrudu/Documents/GitHub/vaccination_rate_GAMs/data')

In [5]:
cases = pd.read_pickle('weekly_cases.pkl')
hospitalized = pd.read_pickle('weekly_hospitalized.pkl')
df = hospitalized.merge(cases)

In [6]:
for fips in df['FIPS'].unique():
    df.loc[df[df['FIPS'] == fips].index, ['Cases']] =\
    df[df['FIPS'] == fips][['Cases']].rolling(window = 3).mean()
df = df.dropna().reset_index().drop(columns = ['index'])
#### Add lag
for fips in df['FIPS'].unique():
    df.loc[df[df['FIPS'] == fips].index, ['Cases_lag1']] =\
    df[df['FIPS'] == fips]['Cases'].shift(1)
df = df.dropna().reset_index().drop(columns = ['index'])
df['hospitalization_rate'] = df['hospitalized']/df['Cases_lag1']

####
for fips in df['FIPS'].unique():
    df.loc[df[df['FIPS'] == fips].index, ['hospitalization_rate']] =\
    df[df['FIPS'] == fips][['hospitalization_rate']].rolling(window = 3).mean()
    
#### Population
pop = pd.read_csv('population_us.csv')
pop['FIPS'] = pop['FIPS'].astype(str).str.zfill(5)
pop['FIPS'] = pop['FIPS'].str[:2]
pop = pop[['FIPS', 'Population']].groupby(['FIPS']).sum().reset_index()
df = pop.merge(df)

#### Vaccine Data
vaccine = pd.read_pickle('weekly_vaccination.pkl')
df = df.merge(vaccine)
for i in vaccine.columns[2:]:
    df[i] = df[i]/df['Population']
for i in vaccine.columns[2:]:
    df[i] = df[i]/7
    
##### Policy data
policy = pd.read_pickle('weekly_policy.pkl')
df = df.merge(policy)

###### Testing data
test = pd.read_pickle('weekly_testing.pkl')
test.Weekly_testing = test.Weekly_testing.mask(test.Weekly_testing.lt(0),np.nan)
test['Weekly_testing'] = test['Weekly_testing'].interpolate(method='linear')
df = df.merge(test)
for i in test.columns[2:]:
    df[i] = df[i]/df['Population']
    
###### Mobility
mobility = pd.read_pickle('weekly_mobility.pkl')
df = df.merge(mobility, how = 'left')
    
#####Previous infection
previous_infection = pd.read_pickle('weekly_previous_infection.pkl')
df = df.merge(previous_infection)

###Genomic_data
genomic = pd.read_pickle('weekly_genomic.pkl')
genomic['Week_Mon'] = pd.to_datetime(genomic['Week_Mon'])
genomic['Dominant_variant'] = genomic['Dominant_variant'].replace('Alpha', 'Pre-Delta')
df = df.merge(genomic)

In [7]:
def create_relative(df, col):
    df_ref = df[['Week_Mon', col]].groupby(['Week_Mon']).mean().reset_index()
    df_ref = df_ref.rename(columns = {col : 'ref'})
    df = df.merge(df_ref, left_on = ['Week_Mon'], right_on = ['Week_Mon'])
    df[col] = df[col]/df['ref']
    df = df.drop(columns = ['ref'])
    df = df.rename(columns = {col : 'Relative_' + col})
    return df

In [8]:
for col in ['People_at_least_one_dose',
       'People_fully_vaccinated', 'Total_additional_doses',
       'GovernmentResponseIndex_Average', 'Weekly_testing',
       'limited_service_restaurant', 'elementary_school',
       'other_general_store', 'gas_station', 'gym', 'cafes_snack_bar',
       'religious_organization', 'parks', 'hardware_store', 'department_store',
       'child_day_care', 'office_of_physician', 'pharmacies',
       'sporting_goods_store', 'automotive_store', 'used_merchandise_store',
       'convenience_store', 'grocery_store', 'full_service_restaurant',
       'hotel', 'University']:
    for fips in df['FIPS'].unique():
        df.loc[df[df['FIPS'] == fips].index, col + '_lag2'] =\
        df[df['FIPS'] == fips][col].shift(2)


In [9]:
for col in ['Past_12_week_infections',
       'Past_16_week_infections', 'Past_20_week_infections',
       'Past_24_week_infections']:
    for fips in df['FIPS'].unique():
        df.loc[df[df['FIPS'] == fips].index, col + '_lag4'] =\
        df[df['FIPS'] == fips][col].shift(4)


In [10]:
for fips in df['FIPS'].unique():
    df.loc[df[df['FIPS'] == fips].index, ['hospitalization_rate']] =\
    df[df['FIPS'] == fips][['hospitalization_rate']].rolling(window = 3).mean()

In [11]:
for col in ['hospitalization_rate','People_at_least_one_dose_lag2', 'People_fully_vaccinated_lag2',
       'Total_additional_doses_lag2', 'GovernmentResponseIndex_Average_lag2',
       'Weekly_testing_lag2', 'limited_service_restaurant_lag2',
       'elementary_school_lag2', 'other_general_store_lag2',
       'gas_station_lag2', 'gym_lag2', 'cafes_snack_bar_lag2',
       'religious_organization_lag2', 'parks_lag2', 'hardware_store_lag2',
       'department_store_lag2', 'child_day_care_lag2',
       'office_of_physician_lag2', 'pharmacies_lag2',
       'sporting_goods_store_lag2', 'automotive_store_lag2',
       'used_merchandise_store_lag2', 'convenience_store_lag2',
       'grocery_store_lag2', 'full_service_restaurant_lag2', 'hotel_lag2',
       'University_lag2', 'Past_12_week_infections_lag4',
       'Past_16_week_infections_lag4', 'Past_20_week_infections_lag4',
       'Past_24_week_infections_lag4']:
       df = create_relative(df, col)

In [12]:
df_hospitalization = df[['FIPS', 'Week_Mon', 'Relative_hospitalization_rate',
                         'Relative_People_at_least_one_dose_lag2',
       'Relative_People_fully_vaccinated_lag2',
       'Relative_Total_additional_doses_lag2',
       'Relative_GovernmentResponseIndex_Average_lag2',
       'Relative_Weekly_testing_lag2',
       'Relative_limited_service_restaurant_lag2',
       'Relative_elementary_school_lag2', 'Relative_other_general_store_lag2',
       'Relative_gas_station_lag2', 'Relative_gym_lag2',
       'Relative_cafes_snack_bar_lag2', 'Relative_religious_organization_lag2',
       'Relative_parks_lag2', 'Relative_hardware_store_lag2',
       'Relative_department_store_lag2', 'Relative_child_day_care_lag2',
       'Relative_office_of_physician_lag2', 'Relative_pharmacies_lag2',
       'Relative_sporting_goods_store_lag2', 'Relative_automotive_store_lag2',
       'Relative_used_merchandise_store_lag2',
       'Relative_convenience_store_lag2', 'Relative_grocery_store_lag2',
       'Relative_full_service_restaurant_lag2', 'Relative_hotel_lag2',
       'Relative_University_lag2', 'Relative_Past_12_week_infections_lag4',
       'Relative_Past_16_week_infections_lag4',
       'Relative_Past_20_week_infections_lag4',
       'Relative_Past_24_week_infections_lag4','Dominant_variant']]

In [13]:
df_hospitalization = df_hospitalization[(df_hospitalization['Week_Mon'] >= pd.to_datetime('2021-04-19'))&(df_hospitalization['Week_Mon'] \
                    <= pd.to_datetime('2022-02-28'))].reset_index().drop(columns = ['index'])

In [14]:
df_static = pd.read_pickle('/Users/hongrudu/Desktop/submission_file/raw_data/static_variable.pkl')

In [15]:
df_hospitalization = df_hospitalization.merge(df_static)

In [16]:
df_hospitalization = df_hospitalization.rename(columns = {'Week_Mon' : 'Week',
                                                          'Relative_University_lag2' : 'Relative_university_lag2',
                             'Relative_GovernmentResponseIndex_Average_lag2' : 'Relative_Government_Response_Index_lag2',
                      'Weekly_testing_lag2' : 'Relative_Weekly_testing_lag2',
                      'Past_12_week_infections_lag4' : 'Relative_Past_12_week_infections_lag4',
                      'Past_16_week_infections_lag4' : 'Relative_Past_16_week_infections_lag4',
                      'Past_20_week_infections_lag4' : 'Relative_Past_20_week_infections_lag4',
                      'Past_24_week_infections_lag4' : 'Relative_Past_24_week_infections_lag4',
                             'Relative_People_at_least_one_dose_lag2' : 'Relative_partial_vaccination_rate_lag2',
                      'Relative_People_fully_vaccinated_lag2' : 'Relative_completed_primary_series_rate_lag2',
                      'Relative_Total_additional_doses_lag2' : 'Relative_booster_vaccination_rate_lag2'})

In [17]:
df_hospitalization = df_hospitalization[['FIPS', 'Week', 'Relative_hospitalization_rate',
       'Relative_university_lag2', 'Relative_full_service_restaurant_lag2',
       'Relative_hotel_lag2', 'Relative_grocery_store_lag2',
     'Relative_limited_service_restaurant_lag2',
       'Relative_elementary_school_lag2', 'Relative_other_general_store_lag2',
       'Relative_gas_station_lag2', 'Relative_gym_lag2',
       'Relative_cafes_snack_bar_lag2', 'Relative_religious_organization_lag2',
       'Relative_parks_lag2', 'Relative_hardware_store_lag2',
       'Relative_department_store_lag2', 'Relative_child_day_care_lag2',
       'Relative_office_of_physician_lag2', 'Relative_pharmacies_lag2',
       'Relative_sporting_goods_store_lag2', 'Relative_automotive_store_lag2',
       'Relative_used_merchandise_store_lag2',
       'Relative_convenience_store_lag2',
     'Relative_Government_Response_Index_lag2',
       'Relative_Weekly_testing_lag2', 'Relative_Past_12_week_infections_lag4',
       'Relative_Past_16_week_infections_lag4',
       'Relative_Past_20_week_infections_lag4',
       'Relative_Past_24_week_infections_lag4',
       'Relative_partial_vaccination_rate_lag2',
       'Relative_completed_primary_series_rate_lag2',
       'Relative_booster_vaccination_rate_lag2', 'Dominant_variant']]

In [18]:
df_hospitalization.to_csv('df_hosp.csv')